In [24]:
import pandas as pd
import seaborn as sns
import re

In [2]:
tsv = pd.read_csv('dnd_chars_tsv.txt', sep='\t')

In [3]:
tsv.head()

,ip,finger,hash,name,race,background,date,class,justClass,subclass,...,country,countryCode,processedAlignment,good,lawful,processedRace,processedSpells,processedWeapons,levelGroup,alias
0,081b9c8,038162c,b36cc828d3d069c3ebf4916c3072480b,bb0cd171,Human,Knight,2021-07-27T01:30:16Z,Warlock 4,Warlock,The Hexblade,...,United States,US,NaN,NaN,NaN,Human,Eldritch Blast*0|Create Bonfire*0|Friends*0|He...,"Crossbow, Light|Dagger|Mace|Longsword",4-7,competent_mclaren
1,081b9c8,038162c,cae2a5212b208dc3c9fdd13c6677662d,bb0cd171,Human,Knight,2021-07-27T01:07:20Z,Warlock 1,Warlock,The Hexblade,...,United States,US,NaN,NaN,NaN,Human,NaN,"Crossbow, Light|Dagger|Mace",1-3,competent_mclaren
2,347f396,ce53022,574c56f949de0adfaa2c4142fb4c54e5,24415eef,Hengenyokai- Sabre Toothed Tiger,Knight,2021-07-26T19:09:58Z,Warlock 3,Warlock,The Great Old One,...,United States,US,NaN,NaN,NaN,Shifter,Chill Touch*0|Eldritch Blast*0|Fire Bolt*0|Mag...,Dagger|Quarterstaff|||,1-3,hardcore_dijkstra
3,0062322,542e778,de4c50489033371f5e2c15ac8d992c08,e9d88d5f,Serpentblood,Knight,2021-07-25T21:15:34Z,Sorcerer 4,Sorcerer,Storm Sorcery,...,Canada,CA,NaN,NaN,NaN,Yaun-Ti,NaN,"Crossbow, Light|Dagger",4-7,zealous_volhard
4,3e52d52,dec07a2,f7a17d3836c7fe22d7fb78d4f2b7d14c,a35cdfac,Half-Elf,Urchin,2021-07-25T18:53:42Z,Rogue 3,Rogue,Swashbuckler,...,United States,US,NaN,NaN,NaN,Half-Elf,NaN,"Rapier|Shortbow|Dagger|Crossbow, Light",1-3,pensive_bhabha


In [4]:
tsv.shape

(9784, 36)

In [5]:
tsv.describe()

,level,HP,AC,Str,Dex,Con,Int,Wis,Cha,good,lawful
count,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,9784.000000,0.0,0.0
mean,4.892069,45.322670,15.406991,12.820830,14.652903,14.346893,11.978945,13.121934,13.143806,NaN,NaN
std,5.140607,92.583394,3.381186,3.988983,3.183303,2.543954,3.160591,3.141482,3.690033,NaN,NaN
min,1.000000,-24.000000,7.000000,1.000000,3.000000,4.000000,0.000000,1.000000,0.000000,NaN,NaN
25%,2.000000,14.000000,14.000000,10.000000,13.000000,13.000000,10.000000,11.000000,10.000000,NaN,NaN
50%,4.000000,31.000000,15.000000,12.000000,15.000000,14.000000,12.000000,13.000000,13.000000,NaN,NaN
75%,6.000000,55.000000,17.000000,16.000000,17.000000,16.000000,14.000000,15.000000,16.000000,NaN,NaN
max,261.000000,6894.000000,222.000000,103.000000,101.000000,103.000000,99.000000,100.000000,99.000000,NaN,NaN


In [6]:
tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9784 entries, 0 to 9783
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ip                  8105 non-null   object 
 1   finger              8867 non-null   object 
 2   hash                9784 non-null   object 
 3   name                9580 non-null   object 
 4   race                9760 non-null   object 
 5   background          9766 non-null   object 
 6   date                9784 non-null   object 
 7   class               9784 non-null   object 
 8   justClass           9784 non-null   object 
 9   subclass            7500 non-null   object 
 10  level               9784 non-null   int64  
 11  feats               3066 non-null   object 
 12  HP                  9784 non-null   int64  
 13  AC                  9784 non-null   int64  
 14  Str                 9784 non-null   int64  
 15  Dex                 9784 non-null   int64  
 16  Con   

## Features

**name**: This column has hashes that represent character names. If the hashes are the same, that means the names are the same. Real names are removed to protect character anonymity. Yes D&D characters have rights.

**race**: This is the race field as it come out of the application. It is not really helpful as subrace and race information all mixed up together and unevenly available. It also includes some homebrew content. You probably want to use the processedRace column if you are interested in this.

**background**: Background as it comes out of the application.

**date**: Time & date of input. Dates before 2018-04-16 are unreliable as some has accidentally changed while moving files around.

**class**: Class and level. Different classes are separated by | when needed.

**justClass**: Class without level. Different classes are separated by | when needed.

**subclass**: Subclasses. Again, separated by | when needed.

**level**: Total character level.

**feats**: Feats chosen by character. Separated by | when needed.

**HP**: Character HP.

**AC**: Character AC.

**Str, Dex, Con, Int, Wis, Cha**: ability scores

**alignment**: Alignment free text field. It is a mess, don't touch it. See processedAlignment,good and lawful instead.

**skills**: List of skills with proficiency. Separated by |.

**weapons**: List weapons. Separated by |. It is somewhat of a mess as it allows free text inputs. See processedWeapons.

**spells**: List of spells and their levels. Spells are separated by |s. Each spell has its level next to it separated by *s. This is a huge mess as its a free text field and some users included things like damage dice in them. See processedSpells.

**day**: A shortened version of date. Only includes day information.

**processedAlignment**: Processed version of the alignment column. Way people wrote up their alignments are manually sifted through and assigned to the matching aligmment. First character represents lawfulness (L, N, C), second one goodness (G,N,E). An empty string means alignment wasn't written or unclear.

**good, lawful**: Isolated columns for goodness and lawfulness.

**processedRace**: I have gone through the way race column is filled by the app and asigned them to correct races. If empty, indiciates a homebrew race not natively supported by the app.

**processedSpells**: Formatting is same as the spells column but it is cleaned up. Using string similarity I tried to match the spells to the full list of spells available in the official publications. The spell is removed if the spell I guessed does not have the correct level or doesn't include all words of the original spell and has too many modifications to be recognizable. It may have a few false matches but it should be mostly fine

**processedWeapons**: Similar to processedSpells, weapons column is matched to the closest official weapon with some restrictions.

**levelGroup**: splits levels into groups as used in the feat percentage plot. Only present in the filtered data but easy enough to make on your own.

## Features to drop for Determining a Character's DND class
- name since it is usually arbitrary and it uses hashes to represent the character names
- race because it is unprocessed data and very messy


In [8]:
tsv.drop(labels = ['ip', 'hash', 'finger', 'name', 'race', 'date', 'class', 'subclass', 'alignment', 'spells', 'choices', 
                    'country', 'countryCode', 'processedAlignment','good', 'lawful', 'alias'], axis=1, inplace=True)

In [9]:
tsv.shape

(9784, 19)

In [10]:
tsv.head()

,background,justClass,level,feats,HP,AC,Str,Dex,Con,Int,Wis,Cha,skills,weapons,castingStat,processedRace,processedSpells,processedWeapons,levelGroup
0,Knight,Warlock,4,Fighting Initiate,29,14,9,16,12,12,8,18,History|Investigation|Deception|Intimidation|P...,"Crossbow, light|Dagger|Mace|Longsword",Cha,Human,Eldritch Blast*0|Create Bonfire*0|Friends*0|He...,"Crossbow, Light|Dagger|Mace|Longsword",4-7
1,Knight,Warlock,1,Fighting Initiate,9,13,10,15,13,12,8,16,History|Investigation|Deception|Intimidation|P...,"Crossbow, light|Dagger|Mace",Cha,Human,NaN,"Crossbow, Light|Dagger|Mace",1-3
2,Knight,Warlock,3,NaN,28,12,11,14,14,11,13,19,Investigation|Nature|Intimidation|Persuasion,Dagger|Quarterstaff|Bite|Claw|Eldritch Blast,Cha,Shifter,Chill Touch*0|Eldritch Blast*0|Fire Bolt*0|Mag...,Dagger|Quarterstaff|||,1-3
3,Knight,Sorcerer,4,NaN,20,11,14,12,12,14,11,16,History|Religion|Insight|Persuasion,"Crossbow, light|Dagger",Cha,Yaun-Ti,NaN,"Crossbow, Light|Dagger",4-7
4,Urchin,Rogue,3,NaN,29,15,8,17,14,10,15,12,Sleight of Hand|Stealth|Investigation|Percepti...,"Rapier|Shortbow|Dagger|Crossbow, Light",Int,Half-Elf,NaN,"Rapier|Shortbow|Dagger|Crossbow, Light",1-3


In [14]:
tsv['skills']

0       History|Investigation|Deception|Intimidation|P...
1       History|Investigation|Deception|Intimidation|P...
2            Investigation|Nature|Intimidation|Persuasion
3                     History|Religion|Insight|Persuasion
4       Sleight of Hand|Stealth|Investigation|Percepti...
                              ...                        
9779    Athletics|Stealth|Animal Handling|Insight|Medi...
9780    Nature|Animal Handling|Insight|Medicine|Percep...
9781            Athletics|History|Intimidation|Persuasion
9782    Acrobatics|History|Investigation|Religion|Insi...
9783        Investigation|Religion|Deception|Intimidation
Name: skills, Length: 9784, dtype: object

In [21]:
skillz = tsv['skills'].to_dict()
skillz

{0: 'History|Investigation|Deception|Intimidation|Persuasion',
 1: 'History|Investigation|Deception|Intimidation|Persuasion',
 2: 'Investigation|Nature|Intimidation|Persuasion',
 3: 'History|Religion|Insight|Persuasion',
 4: 'Sleight of Hand|Stealth|Investigation|Perception|Deception|Persuasion',
 5: 'Religion|Insight|Medicine|Perception|Deception',
 6: 'Arcana|History|Investigation|Insight',
 7: 'Athletics|Sleight of Hand|Arcana|History|Investigation',
 8: 'Acrobatics|Stealth|Intimidation|Performance|Persuasion',
 9: 'Athletics|Stealth|Insight|Perception|Deception|Persuasion',
 10: 'Athletics|Stealth|Insight|Perception|Deception|Persuasion',
 11: 'Stealth|Nature|Religion|Survival|Deception|Intimidation',
 12: 'Sleight of Hand|Stealth|Insight|Perception|Survival|Deception|Intimidation',
 13: 'History|Religion|Insight|Medicine|Persuasion',
 14: 'Arcana|History|Investigation|Insight|Perception',
 15: 'Athletics|Acrobatics|Sleight of Hand|Stealth|Insight',
 16: 'Athletics|Stealth|Nature|P

In [23]:
list(skillz.values())

['History|Investigation|Deception|Intimidation|Persuasion',
 'History|Investigation|Deception|Intimidation|Persuasion',
 'Investigation|Nature|Intimidation|Persuasion',
 'History|Religion|Insight|Persuasion',
 'Sleight of Hand|Stealth|Investigation|Perception|Deception|Persuasion',
 'Religion|Insight|Medicine|Perception|Deception',
 'Arcana|History|Investigation|Insight',
 'Athletics|Sleight of Hand|Arcana|History|Investigation',
 'Acrobatics|Stealth|Intimidation|Performance|Persuasion',
 'Athletics|Stealth|Insight|Perception|Deception|Persuasion',
 'Athletics|Stealth|Insight|Perception|Deception|Persuasion',
 'Stealth|Nature|Religion|Survival|Deception|Intimidation',
 'Sleight of Hand|Stealth|Insight|Perception|Survival|Deception|Intimidation',
 'History|Religion|Insight|Medicine|Persuasion',
 'Arcana|History|Investigation|Insight|Perception',
 'Athletics|Acrobatics|Sleight of Hand|Stealth|Insight',
 'Athletics|Stealth|Nature|Perception|Survival|Intimidation',
 'Athletics|Acrobatics|A

In [ ]:
import PyPDF2

In [ ]:
text = pdf.PdfFileReader('thousand_basic_pregens_v011.pdf')

In [ ]:
print(text)

In [ ]:
out = text.getDocumentInfo()

In [ ]:
out

In [ ]:
f = text.getFields()

In [ ]:
num = text.getPage(0)
num.extractText()